In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import matplotlib as mpl


from holodeck import plot, detstats
import holodeck.single_sources as sings
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

import hasasia.sim as hsim


In [ ]:
SHAPE = None
NREALS = 30
NFREQS = 40
NLOUDEST = 10
RECONSTRUCT = True
NPARS = 21

# Construct Parameter Space

In [ ]:
pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None)

In [ ]:
param_names = pspace.param_names
num_pars = len(param_names)
pars = 0.5*np.ones(num_pars)

In [ ]:
def vary_parameter(
        target_param,    # the name of the parameter, has to exist in `param_names`
        params_list = [0.0,  0.5, 1.0],  # the values we'll check
        pspace = holo.param_spaces.PS_Uniform_09B(holo.log, nsamples=1, sam_shape=SHAPE, seed=None),
        nreals=NREALS, nfreqs=NFREQS,
        pars=None, save_dir=None, 
        get_ds=False, #npsrs=NPSRS, sigma=SIGMA, nskies=NSKIES,
        ):
    # get the parameter names from this library-space
    param_names = pspace.param_names
    num_pars = len(pspace.param_names)
    print(f"{num_pars=} :: {param_names=}")

    # choose each parameter to be half-way across the range provided by the library
    if pars is None:
        pars = 0.5 * np.ones(num_pars) 
    str_pars = str(pars).replace(" ", "_").replace("[", "").replace("]", "")
    # Choose parameter to vary
    param_idx = param_names.index(target_param)

    data = []
    params = []
    dsdat = []
    for ii, par in enumerate(params_list):
        pars[param_idx] = par
        print(f"{ii=}, {pars=}")
        # _params = pspace.param_samples[0]*pars
        _params = pspace.normalized_params(pars)
        params.append(_params)
        # construct `sam` and `hard` instances based on these parameters
        sam, hard = pspace.model_for_params(_params, pspace.sam_shape)
        if isinstance(hard, holo.hardening.Fixed_Time_2PL_SAM):
            hard_name = 'Fixed Time'
        elif isinstance(hard, holo.hardening.Hard_GW):
            hard_name = 'GW Only'
        # run this model, retrieving binary parameters and the GWB
        _data = holo.librarian.run_model(sam, hard, nreals, nfreqs, nloudest=NLOUDEST,
                                        gwb_flag=False, singles_flag=True, params_flag=True, details_flag=True)
        data.append(_data)
        if get_ds:
            _dsdat = detstats.detect_pspace_model(
                _data['fobs_cents'], _data['hc_ss'], _data['hc_bg'], NPSRS, SIGMA, NSKIES)
            dsdat.append(_dsdat)
    if save_dir is not None:
        str_shape = str(sam.shape).replace(", ", "_").replace("(", "").replace(")", "")
        filename = save_dir+'/%s_p%s_s%s.npz' % (target_param, str_pars, str_shape)
        np.savez(filename, data=data, params=params, hard_name=hard_name, shape=sam.shape, target_param=target_param )
        print('saved to %s' % filename)

    return (data, params, dsdat)

# Calibrate PTA

### Get Middle Model

In [ ]:
rv = vary_parameter('hard_time', params_list=[0.5], get_ds=False)
data1 = rv[0][0] # there is o|nly one data being returned
print(f"{len(rv[0])}")

In [ ]:
# calculate dur, cad, dfobs
fobs_edges = data1['fobs_edges']
fobs_cents = data1['fobs_cents']
dur = 1/fobs_cents[0]
print(dur.shape)
hifr = NFREQS/dur
cad = 1.0 / (2 * hifr)

## Change the PTA as needed

In [ ]:
NPSRS = 40
SIGMA = 2.6e-6
NSKIES = 35

# build PTA
print('Building pulsar timing array.')
sum_median=0
sum_mean=0
count=30
# dat = data[int(len(data)/2)]
for ii in range(count):
    phis = np.random.uniform(0, 2*np.pi, size = NPSRS)
    thetas = np.random.uniform(np.pi/2, np.pi/2, size = NPSRS)
    # sigmas = np.ones_like(phis)*sigma
    psrs = hsim.sim_pta(timespan=dur/YR, cad=1/(cad/YR), sigma=SIGMA,
                    phi=phis, theta=thetas)

    dp_bg = detstats.detect_bg_pta(psrs, fobs_cents, cad, data1['hc_bg'])
    # print(f"{np.median(dp_bg)=}, {np.mean(dp_bg)=}, {np.std(dp_bg)=}")
    sum_median += np.median(dp_bg)
    sum_mean += np.mean(dp_bg)
print(f"average median={sum_median/count}, average mean={sum_mean/count}")


In [ ]:
theta_ss, phi_ss, Phi0_ss, iota_ss, psi_ss = detstats._build_skies(NFREQS, NSKIES, NLOUDEST)
dfobs = np.diff(fobs_edges)
hc_ss = data1['hc_ss']
hc_bg = data1['hc_bg']
vals_ss = detstats.detect_ss_pta(
        psrs, cad, dur, fobs_cents, dfobs, hc_ss, hc_bg, 
        gamma_cython=True, snr_cython=True, ret_snr=True, 
        theta_ss=theta_ss, phi_ss=phi_ss, Phi0_ss=Phi0_ss, iota_ss=iota_ss, psi_ss=psi_ss,
        )

In [ ]:
print(NLOUDEST)

In [ ]:
print(theta_ss.shape)
print(hc_ss.shape)
print(hc_bg.shape)
print(data1['hc_bg'].shape)

In [ ]:
print(data1['hc_bg'].shape)

In [ ]:
dfobs = np.diff(fobs_edges)
print(cad.shape)
theta_ss = np.random.uniform(0, np.pi, size = NFREQS * NSKIES * NLOUDEST).reshape(NFREQS, NSKIES, NLOUDEST)
dp_ss, snr_ss, gamma_ssi = detstats.detect_ss_pta(psrs, cad=cad, dur=dur, fobs=fobs_cents, dfobs=dfobs, #theta_ss=theta_ss,
                                                  hc_ss=data1['hc_ss'], hc_bg=data1['hc_bg'], ret_snr=True)

In [ ]:
# check that rest make sense

dsdat1 = detstats.detect_pspace_model(fobs_cents, data1['hc_ss'], data1['hc_bg'], NPSRS, SIGMA, NSKIES)
print(holo.utils.stats(dsdat1['ev_ss']))
print(holo.utils.stats(dsdat1['dp_ss']))
print(holo.utils.stats(dsdat1['ev_ss']-dsdat1['dp_ss']))
print(holo.utils.stats(dsdat1['dp_bg']))

# Run Many Hard Times

In [ ]:
filename = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/anatomy_09B'
name = '/hardtime_%dvariations' % NPARS
filename = filename +name+'.npz'

In [ ]:
RECONSTRUCT=True
if RECONSTRUCT:
    params_list = np.linspace(0,1,NPARS)
    print(params_list)
    data, params, dsdat = vary_parameter('hard_time', params_list=params_list, get_ds=True)
    np.savez(filename, data=data, params=params, dsdat=dsdat)
else:
    file = np.load(filename, allow_pickle=True)
    print(file.files)
    data = file['data']
    params = file['params']
    dsdat = file['dsdat']

## or just reconstruct detstats

In [ ]:
JUST_DETSTATS = False
if JUST_DETSTATS:
    dsdat = []
    for ii, _data in enumerate(data):
        _dsdat = detstats.detect_pspace_model(
            _data['fobs_cents'], _data['hc_ss'], _data['hc_bg'], NPSRS, SIGMA, NSKIES)
        dsdat.append(_dsdat)
    np.savez(filename = filename +'/hardtime_%dvars_p%d_sig%.2e.npz' % (NPARS, NPSRS, SIGMA,),
             data = data, dsdat=dsdat, params=params)

In [ ]:
for ds in dsdat[:3]:
    print(f"{holo.utils.stats(ds['dp_bg'])=}")
    print(f"{holo.utils.stats(ds['dp_ss'])=}")
    print(f"{holo.utils.stats(ds['ev_ss'])=}")

In [ ]:
for ds in dsdat:
    print(holo.utils.stats(ds['dp_bg']/ds['dp_ss'][:,5]))

In [ ]:
# def_draw_ev(ax):
# plt.rcParams['mathtext.fontset'] = "cm"
# plt.rcParams["font.family"] = "serif"

def draw_skies_vs_bg(ax, skies_ss, dp_bg, label=None,
                     color='k', mean=True):
    xx = dp_bg # shape (R,)
    if mean:
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    else: 
        yy = np.mean(skies_ss, axis=-1) # shape (R,)
    yerr = np.std(skies_ss, axis=-1) # shape (R,)

    hh = ax.errorbar(xx, yy, yerr, color=color, label=label,
                linestyle='', capsize=3, marker='o', alpha=0.5)
    return hh


# def plot_evss_vs_dpbg(dsdat):
#     colors = cm.rainbow_r(np.linspace(0, 1, len(dsdat)))

#     fig, ax = plot.figax(xlabel='Background Detection Probability', 
#                          ylabel='$\langle$ Single Source Detections $\\rangle_\mathrm{skies}$')
#     for ii, ds in enumerate(dsdat):
#         draw_skies_vs_bg(ax, ds['ev_ss'], ds['dp_bg'], color=colors[ii])
#     # xx = fobs_cents*YR
#     # yy = dsdat['ev_ss']

#     return fig

def plot_dpss_vs_dpbg(dsdat, params, target_param='hard_time', use_ev=True):
    colors = cm.rainbow_r(np.linspace(0, 1, len(dsdat)))
    sslabel = 'Expected Number' if use_ev else 'Detection Probability'
    fig, ax = plot.figax(xlabel='Background Detection Probability', 
                         ylabel='$\langle$ Single Source %s} $\\rangle_\mathrm{skies}$' % sslabel,
    ) #xscale='linear', yscale='linear')
    handles = []
    for ii, ds in enumerate(dsdat):
        label = '%.2f' % params[ii][target_param]
        detss = ds['ev_ss'] if use_ev else ds['dp_ss']
        hh = draw_skies_vs_bg(ax, detss, ds['dp_bg'], color=colors[ii], label=label)
        handles.append(hh)
    ax.legend(handles=handles, loc = 'lower left', title='$\\tau_\mathrm{hard}$',
              ncols=4, title_fontsize=12)
    # ax.set_aspect(1)
    # xx = fobs_cents*YR
    # yy = dsdat['ev_ss']

    fig.tight_layout()
    return fig

text = 'GSMF: $\psi_0=%.2f, m_{\phi,0}=%.2f$' % ((params[0]['gsmf_phi0']), (params[0]['gsmf_mchar0_log10']))
text = text+'\nMMB: $\mu = %.2f, \epsilon_\mu=%2f$ dex' % (params[0]['mmb_mamp_log10'], params[0]['mmb_scatter_dex'])
text = text+'\n$da/dt: \gamma_\mathrm{inner}=%.2f, \gamma_\mathrm{outer}=%.2f$' % (params[0]['hard_gamma_inner'], 2.5)
print(text)

param_path = '/Users/emigardiner/GWs/holodeck/output/figures/params'


fig1 = plot_dpss_vs_dpbg(dsdat, params, use_ev=True)
fig1.axes[0].text(0.99,0.99, text, transform=fig1.axes[0].transAxes, verticalalignment='top', horizontalalignment='right')
fig1.tight_layout()
fig1.savefig(param_path+'/')

fig2 = plot_dpss_vs_dpbg(dsdat, params, use_ev=False)
fig2.axes[0].text(0.99,0.99, text, transform=fig1.axes[0].transAxes, verticalalignment='top', horizontalalignment='right')
fig2.tight_layout()

In [ ]:

xlabel='$\\tau_\mathrm{hard}$'
ylabel='$\gamma_\mathrm{SS}/\gamma_\mathrm{BG}$'
colors = cm.rainbow_r(np.linspace(0, 1, len(params)))


fig, ax = plot.figax(xscale='linear',)
ax.set_ylabel(ylabel, fontsize=14)
ax.set_xlabel(xlabel, fontsize=14)

target='hard_time'
xx = []
yy = []
for ii, par in enumerate(params):
    xx.append(params[ii][target])
    dp_bg = dsdat[ii]['dp_bg']
    dp_ss = np.mean(dsdat[ii]['dp_ss'], axis=-1)
    yy.append(dp_ss/dp_bg)
    for rr in range(len(yy[0])):
        ax.scatter(xx[ii], yy[ii][rr], color=colors[ii], alpha=0.5)
xx = np.array(xx)
yy = np.array(yy)
print(yy.shape)
# for rr, yi in enumerate(yy):
#     ax.scatter(xx,yy[:,rr])
plot.draw_med_conf(ax, xx, yy)
# for rr in range(len(yy[0])):
#     ax.scatter(xx,yy[:,rr], alpha=0.5, color=colors[])

Could do the same, but vary another model parameter for color, or show some other parameter

In [ ]:
# Get mass normalization

sspar = []
ssmtot = []
for dat in data:
    sspar.append(dat['sspar'])
    ssmtot.append(dat['sspar'][0])
sspar = np.array(sspar)
ssmtot = np.array(ssmtot)/MSOL
ssmtot = np.max(ssmtot, axis=(1,3))
norm = mpl.colors.Normalize(vmin=np.min(ssmtot), vmax=np.max(ssmtot))

print(np.min(ssmtot))
print(np.max(ssmtot))
print(ssmtot.shape)

In [ ]:
print(norm(1e43))
print(norm(0))
cmap = cm.viridis
print(cmap(0.5))

In [ ]:
xlabel='$\\tau_\mathrm{hard}$'
ylabel='$\gamma_\mathrm{SS}/\gamma_\mathrm{BG}$'
cmap = cm.gist_rainbow

fig, ax = plot.figax(xscale='linear',)
ax.set_ylabel(ylabel, fontsize=14)
ax.set_xlabel(xlabel, fontsize=14)

target='hard_time'
xx = []
yy = []
for ii, par in enumerate(params):
    xx.append(params[ii][target])
    dp_bg = dsdat[ii]['dp_bg']
    dp_ss = np.mean(dsdat[ii]['dp_ss'], axis=-1)
    yy.append(dp_ss/dp_bg)
    for rr in range(len(yy[0])):
        ax.scatter(xx[ii], yy[ii][rr], color=cmap(norm(ssmtot[ii,rr])),  alpha=0.5,)
xx = np.array(xx)
yy = np.array(yy)
print(yy.shape)
# for rr, yi in enumerate(yy):
#     ax.scatter(xx,yy[:,rr])
plot.draw_med_conf(ax, xx, yy)
# for rr in range(len(yy[0])):
#     ax.scatter(xx,yy[:,rr], alpha=0.5, color=colors[])
cbar = fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap), label='Mass of Loudest Source [M$_\odot$]')